In [1]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
# def clean_lyrics(string,xWords):
#     for word in xWords:
#         string = string.replace(word, '')
#     return string

# xWords = ['[Intro]','[Verse 1]','[Verse 2]', '[Pre-Chorus]', '[Chorus]', '[Post-Chorus]']
# clean_lyrics = clean_lyrics(lyrics.text, xWords)
# print(clean_lyrics, '\n')

# unnecessary_words = ['a', 'an', 'and', 'the', 'of']
# print(' '.join(i for i in clean_lyrics.split() if i not in unnecessary_words))

In [54]:
import string
import re
chars = re.escape(string.punctuation)

song = "Signed, Sealed, Delivered (I'm Yours)"
songname = re.sub(r'['+chars+']', '', (song.split(' - ')[0].split("[")[0].split(",")[0]))
clean_songname = songname.replace(' ','-') if ' ' in songname else songname
songname_url = clean_songname[:-1] if clean_songname[-1:] == '-' else clean_songname

print(songname_url)

print(chars)

Signed
!"\#\$%\&'\(\)\*\+,\-\./:;<=>\?@\[\\\]\^_`\{\|\}\~


In [59]:
song = 'Frank Ocean: Pink + White'
songname = song
for string in ["(Pt. 1 & Pt. 2)", "(Part 1)", "(Part 2)", "(Live)", "(feat)", "(feat. ", "(Theme", "(with", "- Remaster", "+ "]:
    if string in song:
        songname = song.split(string)[0][:-1]
songname_2 = re.sub(r'['+chars+']', '', songname)

# clean_songname = songname.replace(' ','-') if ' ' in songname else songname
# songname_url = clean_songname[:-1] if clean_songname[-1:] == '-' else clean_songname

# (feat. xxx)
print(songname_2)

# 

Frank Ocean Pink


In [ ]:
sonFrank Ocean: Pink + White 